In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\riswa\\Desktop\\AI\\Text-Summarizer'

In [4]:
from src.summarizer.logging import logging
from src.summarizer.exception import CustomException
from dataclasses import dataclass
from pathlib import Path
from src.summarizer.constants import  *
from src.summarizer.utils import *



In [21]:
# @dataclass(frozen= True)
# class DataIngestionConfig:
#     root_dir : Path
#     local_data_file : Path
#     source_URL : str
#     unzip_dir: Path
    
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
class ConfigManager:

    def __init__(self,config_path = CONFIG_PATH,params_path = PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            local_data_file = config.local_data_file,
            source_URL = config.source_URL,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
    
    
    

In [23]:
import os
import urllib.request as request
import zipfile
from summarizer.logging import logger
from summarizer.utils  import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
import sys
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[2024-08-09 20:29:10,044: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-09 20:29:10,049: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-09 20:29:10,052: INFO: __init__: created directory at: artifacts]
[2024-08-09 20:29:10,055: INFO: __init__: created directory at: artifacts/data_ingestion]
[2024-08-09 20:29:22,776: INFO: __init__: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F72:22AFD1:5A046D:726E7A:66B62EC5
Accept-Ranges: bytes
Date: Fri, 09 Aug 2024 14:59:18 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10227-MAA
X-Ca